In [31]:
#Imports
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

from chromadb.utils import embedding_functions

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from scipy.cluster.hierarchy import dendrogram, linkage
from dotenv import load_dotenv
from typing import List, Union
import matplotlib.pyplot as plt
import os, re, time, random
import networkx as nx
import pandas as pd
import numpy as np
import nltk
from nltk import sent_tokenize

import pathlib
import chromadb

import dspy
import dsp
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [32]:
# Para comprobar la conexión al servidor LLaMA
!curl -X POST http://127.0.0.1:8090/generate -d '{"prompt": "Hello"}'

Expected request with `Content-Type: application/json`

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
###########
#   LLM   #
###########
path_env = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env")
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

lm = dspy.HFClientTGI(model="meta-llama/meta-llama-3-8b-instruct", port=8090, url="http://127.0.0.0") #"meta-llama/Meta-Llama-3-8B "
#lm = dspy.OpenAI(model="gpt-3.5-turbo")# "gpt-4o-2024-05-13")

path_env = pathlib.Path("/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env")
print(path_env)
load_dotenv(path_env)
api_key = os.getenv("HUGGINGFACE_API_KEY")
os.environ["HUGGINGFACE_API_KEY"] = api_key

dspy.settings.configure(lm=lm, temperature = 0)

/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env
/export/usuarios_ml4ds/cggamella/NP-Search-Tool/.env


# Detección de acrónimos con dspy  (Etapa 1)

In [34]:
ruta_archivo = '/export/usuarios_ml4ds/cggamella/RAG_tool/acronyms_paper.xlsx'
df = pd.read_excel(ruta_archivo)

In [35]:
df.columns

Index(['acronyms', 'manual_expanded_acronyms', 'text',
       'detected_acronyms_LLaMA', 'expanded_LLaMA', 'expanded'],
      dtype='object')

In [36]:
df.head(2)

,acronyms,manual_expanded_acronyms,text,detected_acronyms_LLaMA,expanded_LLaMA,expanded
0,ONG,Organización no gubernamental,la ong ha lanzado una campaña de concienciació...,ONG,Organización No Gubernamental,NaN
1,PCAP,Pliego de cláusulas administrativas particulares,el contrato de obra pública se regirá por el p...,PCAP,pliego de cláusulas administrativas particulares,NaN


In [37]:
class AcronymDetector(dspy.Signature):
    """
    Detecta los acrónimos, abreviaturas y siglas que contenga el texto.

    TEXTO: La reforma del sistema tributario ha traído cambios significativos en el IVA,
    afectando tanto a consumidores como a empresarios. Las PYME han mostrado preocupación por el aumento
    de la carga fiscal, ya que sus márgenes de beneficio son más ajustados. Por otro lado, el gobierno ha
    ajustado las tasas del IRPF para aliviar la presión sobre las rentas más bajas. Según los últimos informes,
    el PIB del país ha crecido un 2% en el último trimestre, reflejando una recuperación económica gradual.
    Sin embargo, los expertos advierten que es esencial seguir monitorizando estos indicadores para evitar futuros desequilibrios.
    ACRONIMOS:IVA, PYME, IRPF, PIB
    """

    TEXTO = dspy.InputField(desc="Texto en español que puede contener o no, acrónimos, siglas y/o abreviaturas que deben detectarse")
    ACRONIMOS = dspy.OutputField(desc="Lista de acrónimos, siglas y/o abreviaturas. Un acrónimo, es un tipo de palabra formada a partir de la fusión de varias palabras.")

class AcronymDetectorModule(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generator = dspy.Predict(AcronymDetector)

    def forward(self, texto):
        response = self.generator(TEXTO=texto)
        return dspy.Prediction(ACRONIMOS=response.ACRONIMOS)

In [ ]:
# Crear una instancia del módulo
my_acronym_detector = AcronymDetectorModule()

# Lista para almacenar los acrónimos detectados
acronyms_detected = []

# Detectar acrónimos en cada texto del DataFrame
for index, row in df.iterrows():
    print(f"Processing row {index}: {row['text']}")
    prediction = my_acronym_detector.forward(row['text'])
    print(f"Detected acronyms: {prediction.ACRONIMOS}")
    acronyms_detected.append(prediction.ACRONIMOS)

In [11]:
# Guardar los acrónimos detectados
df['detected_acronyms_LLaMA'] = acronyms_detected

In [ ]:
lm.inspect_history(4)

# Nueva forma de hacer chunks

In [39]:
# La modifico incluyendo como parámetro el window_overlap para intanciarla con mayor control y ajuste del final del doc
class Chunker:
    def __init__(self, context_window=3000, max_windows=5, window_overlap = 0.02):
        self.context_window = context_window
        self.max_windows = max_windows
        self.window_overlap = window_overlap

    def __call__(self, paper):
        snippet_idx = 0
        startpos = 0

        while snippet_idx < self.max_windows and len(paper) > startpos:
            endpos = startpos + int(self.context_window * (1.0 + self.window_overlap))
            if endpos > len(paper):
                endpos = len(paper)
            snippet = paper[startpos:endpos]

            next_newline_pos = snippet.rfind('\n')
            if next_newline_pos != -1 and next_newline_pos >= self.context_window // 2:
                snippet = snippet[:next_newline_pos + 1]

            yield snippet_idx, snippet.strip()
            startpos += self.context_window - int(self.context_window * self.window_overlap)
            snippet_idx += 1

In [14]:
chunker = Chunker(context_window=3000, max_windows=100, window_overlap = 0.1)

In [ ]:
# Normalizar el campo textual para evitar problemas
df['text'] = df['text'].str.lower()

documents = []

for idx, row in df.iterrows():
    example_document = row['text']
    print("EXAMPLE DOC:\n",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial"})
    print("El DOC es:\n",doc)

    # Dividir el documento en fragmentos usando la instancia de Chunker
    for _, chunk in chunker(doc.page_content):
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)

In [ ]:
documents

In [ ]:
'''
# Normalizar el campo textual para evitar problemas
df['text'] = df['text'].str.lower()
# Crear el text splitter, [chunk_size: #caracteres de cada chunk];
#[chunk_overlap: #caracteres solapan entre chunks para no perder info.]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap=200)

# Crear una lista para almacenar los documentos
documents = []
# Procesar cada fila como un documento separado
for idx, row in df.iterrows():
    example_document = row['text']
    print("el example doc es:",example_document)
    doc = Document(page_content=example_document, metadata={"url": "local", "source": "initial"}) #, "identifier": row['identifier']})
    print("El doc es:",doc)
    # Dividir el documento en fragmentos
    chunks = text_splitter.split_text(doc.page_content)
    print("Los chunks son:",chunks)
    for chunk in chunks:
        chunk_doc = Document(page_content=chunk, metadata=doc.metadata)
        print("EL chunk_doc es:\n",chunk_doc)
        documents.append(chunk_doc)
'''

# Create database

In [ ]:
# Embed and store the texts
path_to_index = '/export/usuarios_ml4ds/cggamella/RAG_tool'
# Supplying a persist_directory will store the embeddings on disk
persist_directory = (pathlib.Path(path_to_index) / 'db_POC_acronyms').as_posix()

In [ ]:
# Calcular los embeddings con un modelo de uso libre de HuggingFace
# Using HuggingFace models
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key="HUGGINGFACE_API_KEY",
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
embeddings = HuggingFaceEmbeddings(model_name)

start = time.time()
# Almacenar los fragmentos en Chroma
# Se extrae el contenido (page_content).El contenido se pasa al modelo de embeddings.
# El vector resultante se almacena en la base de datos junto con los metadatos(índices).

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=model,
    persist_directory=persist_directory
)
end = time.time()
print(f"Total time is {end - start} seconds")

### Create Chroma db without Langchain

In [50]:
import chromadb

path_to_index = '/export/usuarios_ml4ds/cggamella/RAG_tool/example1'

In [51]:
class CustomEmbeddings():

    def __init__(self, model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
        self.model = SentenceTransformer(model)
        
    def encode(self, texts):
        return self.model.encode(texts)

    def __call__(self, input):
        return self.encode(input)
    
embedding_model = CustomEmbeddings('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
client = chromadb.PersistentClient(path=path_to_index)

In [52]:
collection = client.get_or_create_collection(name="test1", embedding_function=embedding_model)

In [ ]:
chunker = Chunker(context_window=3000, max_windows=100, window_overlap = 0.1)

def addVectorDataToDb(df, chunker) -> None:
    embeddings: list = []
    metadatas: list = []
    documents: list = []
    ids: list = []
    
    df['text'] = df['text'].str.lower() # Normalizar el campo textual para evitar problemas
    
    try:
        for idx, row in df.iterrows():      
            # divide in chunks
            for id_chunk, chunk in chunker(row.text):
                embeddings.append(embedding_model.encode(chunk).tolist())
                #metadatas.append(f"{str(idx)}_{str(id_chunk)}")
                documents.append(chunk)
                ids.append(f"{str(idx)}_{str(id_chunk)}")
        collection.add(
            embeddings=embeddings,
            #metadatas=metadatas,
            documents=documents,
            ids=ids
        )
        print("Data added to collection")
    except Exception as e:
        print("Add data to db failed : ", e)
        
addVectorDataToDb(df, chunker)

In [55]:
def searchDataByVector(query: str):
    try:
        query_vector = embedding_model.encode(query).tolist()
        res = collection.query(
            query_embeddings=[query_vector],
            n_results=1,
            include=['distances','embeddings', 'documents'],
        )
        print("Query", "\n--------------")
        print(query)
        print("Result", "\n--------------")
        print(res['documents'][0][0])
        print("Vector", "\n--------------")
        print(res['embeddings'][0][0])
        print("")
        print("")
        print("Complete Response","\n-------------------------")
        print(res)

    except Exception as e:
        print("Vector search failed : ", e)

In [ ]:
example_pdf_1 = "El PDF, formato de documento portátil, se ha convertido en un estándar para la distribución de documentos en línea. Permite a los usuarios compartir información de manera segura y profesional, preservando el formato original independientemente del software o dispositivo utilizado para su visualización."
example_pdf_2 = "En el campo de la ingeniería, es crucial para evaluar la fiabilidad de estructuras y materiales. Ayuda a los ingenieros a modelar y prever el comportamiento de sistemas complejos mediante el análisis de la distribución de probabilidades. Desde la planificación de proyectos hasta la gestión de riesgos, la PDF proporciona herramientas poderosas para optimizar el diseño y la operación de infraestructuras críticas."

searchDataByVector(query="formato de documento portátil")

# Cargar la bbdd ya creada

In [ ]:
vectordb = Chroma(persist_directory=persist_directory)

In [ ]:
results = vectordb.get(limit=1, include=['documents', 'embeddings'])

In [ ]:
query_vector = embedding_model.encode(query).tolist()
res = collection.query(
    query_embeddings=[query_vector],
    n_results=1,
    include=['distances','embeddings', 'documents', 'metadatas'],
)

In [ ]:
results

# Custom Retrieval Module

In [67]:
class AcronymAwareRetriever(dspy.Retrieve):
    """
    Custom retriever for searching documents containing a specific acronym.
    """
    def __init__(self, vectordb, k:int = 3):
        super().__init__(k=k)
        self.vectordb = vectordb

    def forward(self, acronym: Union[str, List[str]], k: int = None) -> dspy.Prediction:
        """
        Retrieve documents containing the provided acronym by directly querying
        the vector database. The documents are filtered to include only those
        where the acronym appears as a standalone word.
        """
        # Creating regex to match the acronym as a standalone word
        regex = r'\b' + re.escape(acronym) + r'\b'

        # Retrieve documents that might contain the acronym
        results = self.vectordb.get(where_document={"$contains": acronym}, limit=k if k else self.k,
                                    include=['documents', 'embeddings'])

        # Extract relevant information
        documents = results.get('documents', [])
        embeddings = results.get('embeddings', [])

         # Filter documents and their embeddings using regex, avoiding duplicates
        seen_documents = set()
        filtered_documents = []
        filtered_embeddings = []
        filtered_passages = []

        for doc, emb in zip(documents, embeddings):
            # Check if the document contains the acronym as a standalone word and is not already added
            if re.search(regex, doc, re.IGNORECASE) and doc not in seen_documents:
                seen_documents.add(doc)
                filtered_documents.append(doc)
                filtered_embeddings.append(emb)
                # Extract passages for documents that pass the regex check
                filtered_passages.append(self.extract_passages(doc, acronym))


        return dspy.Prediction(
            documents=filtered_documents,
            embeddings=filtered_embeddings,
            passages=filtered_passages
        )

    def extract_passages(self, text, acronym):
        """
        Extract the two preceding and two following sentences after the first appearance of the acronym in
        the text.
        """
        sentences = sent_tokenize(text)
        for i, sentence in enumerate(sentences):
            if acronym.lower() in sentence.lower():
                start = max(i - 2, 0)
                end = min(i + 3, len(sentences))
                return ' '.join(sentences[start:end])
        return ""

# Desambiguación de acrónimos con dspy (Etapa 2)

In [76]:
class AcronymExpander(dspy.Signature):
    """
    Expande los acrónimos basándose en el contexto del texto.

    TEXTO: Las antenas que operan en la banda de los 60 Ghz se dice que funcionan en la banda de ondas milimétricas, pues bien su longitud de onda está en el orden de los milímetros.

    ACRONIMO: ghz
    EXPANSION: gigahercio
    """

    TEXTO = dspy.InputField(desc="Texto que contiene el acrónimo,sigla o abreviatura y aporta información sobre la forma expandida")
    ACRONIMO = dspy.InputField(desc="Acrónimo que necesita ser expandido")
    EXPANSION = dspy.OutputField(desc="Es la forma expandida del acrónimo")

class AcronymExpanderModule(dspy.Module):
    def __init__(self):
        super().__init__()
        #self.expander = dspy.Predict(AcronymExpander)
        self.expander = dspy.ChainOfThought(AcronymExpander)

    def forward(self, texto, acronimo):
        response = None
        try:
            response = self.expander(TEXTO=texto, ACRONIMO=acronimo)
            print(f"En el forward la respuesta para el texto '{texto}' es:{response}")
        except Exception as e:
            print(f"-- -- Error expanding acronym: {e}")
            response = None

        return dspy.Prediction(
            texto=texto,
            acronimo=acronimo,
            expansion=response.EXPANSION if response else None
        )

In [77]:
# Uso del módulo
module = AcronymExpanderModule()
result = module.forward(texto="Ejemplo de texto", acronimo="Ej")

En el forward la respuesta para el texto 'Ejemplo de texto' es:Prediction(
    rationale='${produce the EXPANSION}. We...',
    EXPANSION='Ejemplo de texto'
)


# Disambiguation logic

In [78]:
vectordb=collection

In [79]:
acronym_retriever = AcronymAwareRetriever(vectordb=vectordb, k=5)
acronym_expander = AcronymExpanderModule()

In [81]:
# Creo nueva columna en el df para guardar los acrónimos expandidos
df['expanded_LLaMA'] = 'None'

for index, row in df.iterrows():
    text = row['text']
    # Los acrónimos están separados por comas si se detecta más de uno.
    detected_acronyms = str(row['detected_acronyms_LLaMA']).lower().split(',')
    print(detected_acronyms)
    detected_acronyms = [acronym.strip() for acronym in detected_acronyms if acronym.strip() not in ['', 'n/a', 'none', 'ninguno']]

    # Lista para expansiones de acrónimos encontrados
    expansions = []

    for acronym in detected_acronyms:
        # Recuperar documentos relevantes utilizando el retriever
        retrieved_docs = acronym_retriever.forward(acronym).passages

        # Concatenar todos los docs retrieval
        concatenated_docs = " ".join(retrieved_docs)
        # Usar docs concatenados
        if concatenated_docs:
            expansion_result = acronym_expander.forward(concatenated_docs, acronym)
            expansions.append(expansion_result.expansion)

    # Guardar las expansiones en la columna 'expanded_LLaMA'
    df.at[index, 'expanded_LLaMA'] = ', '.join(expansions)

['ong']
En el forward la respuesta para el texto 'la ong ha lanzado una campaña de concienciación sobre la importancia de la protección del medio ambiente. la organización no gubernamental ha trabajado en colaboración con diferentes entidades para promover la sostenibilidad y la conservación de la biodiversidad. gracias al apoyo de voluntarios y donantes, la ong ha logrado implementar proyectos de reforestación y limpieza de playas en todo el país.' es:Prediction(
    rationale='${produce the EXPANSION}. We...',
    EXPANSION='organización no gubernamental'
)
['pcap']
En el forward la respuesta para el texto 'el contrato de obra pública se regirá por el pliego de cláusulas administrativas particulares (pcap), el cual establece las condiciones específicas que deben cumplir tanto la administración como la empresa contratista. es fundamental revisar detenidamente el pcap para garantizar el cumplimiento de todas las obligaciones y evitar posibles conflictos durante la ejecución del proyect

# Integración de grafos de conocimiento para desambiguación de contextos

In [ ]:
class GraphVisualizer:
    def __init__(self, kernel='umbral', threshold=0.21, remove_self_links=True):
        self.kernel = kernel
        self.threshold = threshold
        #self.symmetric = symmetric
        self.remove_self_links = remove_self_links

    def set_embeddings(self, embeddings):
        self.embeddings = np.array(embeddings)

    def _get_similarity_matrix(self):
        if self.embeddings is None:
            raise ValueError("Please set embeddings before trying to get similarity matrix.")
        #print(type(self.embeddings), self.embeddings.shape)
        print(self.embeddings)
        similarity_matrix = cosine_similarity(self.embeddings)
        print("La matriz de similitud es:\n", similarity_matrix)
        if self.kernel == "umbral":
            similarity_matrix[similarity_matrix < self.threshold] = 0
        if self.remove_self_links:
            np.fill_diagonal(similarity_matrix, 0)
        return similarity_matrix

    def visualize_graph(self, plot_graph=True):
        similarity_matrix = self._get_similarity_matrix()
        G = nx.from_numpy_array(similarity_matrix, create_using=nx.DiGraph)

        if plot_graph:
            pos = nx.spring_layout(G, seed=42)
            nx.draw(G, pos, node_color='skyblue', with_labels=True, node_size=70, edge_color='blue', width=0.4)
            plt.title("Document Similarity Graph")
            plt.show()

        return {
            'number_of_nodes': G.number_of_nodes(),
            'number_of_edges': G.number_of_edges(),
            'edges_per_node': G.number_of_edges() / G.number_of_nodes()
        }

In [ ]:
result_predict = acronym_retriever.forward("pdf")


# Instanciar y configurar GraphVisualizer
visualizer = GraphVisualizer(kernel='umbral', threshold=0.31, remove_self_links=True)
visualizer.set_embeddings(result_predict.embeddings)

# Visualizar el grafo de similitud
visualizer.visualize_graph()

# Crear dataset de entrenamiento y validación

In [ ]:
'''
def create_dtset_expanded_acronyms(excel_path):
    # Cargar datos
    df = pd.read_excel(excel_path)
    df = df[['acronyms', 'manual_expanded_acronyms', 'text']]

    # Dividir los datos en conjuntos de entrenamiento y validación
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Convertir los df de train y dev en listas de diccionarios
    data_train = train_df.to_dict('records')
    data_test = test_df.to_dict('records')

    # Create train examples
    trainset = [
        dspy.Example({'texto': row['text'], 'acronimos': row['acronyms'], 'expansion': {row['acronyms']: row['manual_expanded_acronyms']}}) for row in data_train
    ]
    # Create dev examples
    devset = [
        dspy.Example({'texto': row['text'], 'acronimos': row['acronyms'], 'expansion': {row['acronyms']: row['manual_expanded_acronyms']}}) for row in data_test
    ]

    return trainset, devset
'''

In [ ]:
# Cargar los datos
df = pd.read_excel(excel_path)
df = df[['text','manual_expanded_acronyms', 'detected_acronyms_LLaMA','expanded_LLaMA']]

# Dividir los datos en entrenamiento y prueba
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
def create_examples(df):
    examples = []
    for index, row in df.iterrows():
        # Asegurar que ambos campos, acrónimos y expansiones, contienen datos
        if pd.notna(row['detected_acronyms_LLaMA']) and pd.notna(row['expanded_LLaMA']):
            acronyms_list = [acr.strip() for acr in row['detected_acronyms_LLaMA'].split(',')]
            expansions_list = [exp.strip() for exp in row['expanded_LLaMA'].split(',')]

            # Verificar que ambos listados tengan el mismo número de elementos
            if len(acronyms_list) != len(expansions_list):
                print(f"Error en la fila {index}: Número desigual de acrónimos y expansiones.")
                print(f"Fila problemática: {row.to_dict()}")
                continue

            # Crear un ejemplo por cada acrónimo y su correspondiente expansión
            for acr, exp in zip(acronyms_list, expansions_list):
                examples.append(dspy.Example({
                    'texto': row['text'],
                    'acronimo': acr,
                    'expansion_correcta': exp
                }).with_inputs('texto', 'acronimo'))
        else:
            print(f"Datos faltantes en la fila {index}.")
            print(f"Fila problemática: {row.to_dict()}")

    return examples

In [ ]:
train_examples = create_examples(train_df)
test_examples = create_examples(test_df)

In [ ]:
train_examples

In [ ]:
test_examples

In [ ]:
print(len(test_examples))
print(len(train_examples))

In [ ]:
# Cargar el modelo de SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [ ]:
def validate_expansion(example, pred, trace = None):

    prediction_embedding = model.encode(pred.expansion)
    reference_embedding = model.encode(example.expansion_correcta)

    print("La pred embedding es:", prediction_embedding)
    print("La ref embedding es:", reference_embedding)

    # Calcular la similitud del coseno
    similarity = cosine_similarity([prediction_embedding], [reference_embedding])[0][0]

    # Evaluar si la similitud está por encima de un umbral-> [-1,1]
    if similarity > 0.9:
        # Considerando que si está por encima de 0.9 es un acierto
        print(similarity)
        return True
    else:
        print(similarity)
        return False

In [ ]:
# Testing validate_expansion metric
example = type('Example', (object,), {'expansion_correcta': "El PDF, formato de documento portátil, se ha convertido en un estándar para la distribución de documentos en línea. Permite a los usuarios compartir información de manera segura y profesional, preservando el formato original independientemente del software o dispositivo utilizado para su visualización."})
pred = type('Prediction', (object,), {'expansion': "En el campo de la ingeniería, es crucial para evaluar la fiabilidad de estructuras y materiales. Ayuda a los ingenieros a modelar y prever el comportamiento de sistemas complejos mediante el análisis de la distribución de probabilidades. Desde la planificación de proyectos hasta la gestión de riesgos, la PDF proporciona herramientas poderosas para optimizar el diseño y la operación de infraestructuras críticas."})
is_correct = validate_expansion(example, pred)
print("¿Es correcta la expansión?:", is_correct)

In [ ]:
teleprompter = BootstrapFewShotWithRandomSearch(
    metric=validate_expansion,
    max_bootstrapped_demos=4,
    max_labeled_demos=16,
    num_candidate_programs=16,
    max_rounds=1,
)

In [ ]:
# Usar la clase
excel_path = '/export/usuarios_ml4ds/cggamella/RAG_tool/acronyms_paper.xlsx'
# Crear conjuntos de entrenamiento y validación
#trainset, devset = create_dtset_expanded_acronyms(excel_path)

In [ ]:
trainset[1]

In [ ]:
devset[0]

In [ ]:
compiled_model = teleprompter.compile(
    acronym_expander,
    trainset=train_examples,
    valset=test_examples
)